In [67]:
#!pip install mesa
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

In [68]:
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for obj in cell_content:
            if isinstance(obj, RobotLimpieza):
                grid[x][y] = 10
            elif isinstance(obj, Celda):
                grid[x][y] = obj.estado
    return grid

In [69]:
class RobotLimpieza(Agent):
    '''
    Descripcion...
    '''
    def __init__(self, uniqueId, model):
        super().__init__(uniqueId, model)
        self.siguientePos = None
        self.pos = uniqueId
    
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = False)

        # maquina de estados
        for vecino in vecinos:
            if isinstance(vecino, Celda) and self.pos == vecino.pos:
                if vecino.estado == 1:
                    vecino.siguienteEstado = 0
                    self.siguientePos = self.pos
                else:
                    vecinos_2 = self.model.grid.get_neighbors(
                        self.pos,
                        moore = True,
                        include_center = False)
                    vecino.siguienteEstado = 0
                    self.siguientePos = self.random.choice(vecinos_2)
                break

    def advance(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = False)

        for vecino in vecinos:
            if isinstance(vecino, Celda) and self.pos == vecino.pos:
                vecino.estado = vecino.siguienteEstado
                break
        self.model.grid.move_agent(self, self.siguientePos)
        self.pos = self.siguientePos


class Celda(Agent):
    def __init__(self, uniqueId, model, estado):
        super().__init__(uniqueId, model)
        self.pos = uniqueId
        self.estado = estado
        self.siguienteEstado = None
         
class Habitacion(Model):
    def __init__(self, M, N, numAgentes, porcCeldasSucias, tiempoMaximo):
        self.numAgentes = numAgentes
        self.porcCeldasSucias = porcCeldasSucias
        self.porcCeldasLimpias = 1 - porcCeldasSucias
        self.grid = MultiGrid(M,N,False)
        self.schedule = SimultaneousActivation(self)

        # Colocación de celdas limpias/sucias
        numCeldasSucias = int(M*N*porcCeldasSucias)
        for (content, x, y) in self.grid.coord_iter():
            num = random.randint(0,1)
            
            if num == 1 and numCeldasSucias > 0:
                a = Celda((x,y),self,1)
                numCeldasSucias -=1
            else:
                a = Celda((x,y), self, 0)

            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)
        # Colocación de celdas limpias/sucias
        for id in range(numAgentes):
            r = RobotLimpieza(id,self)
            self.grid.place_agent(r, (1, 1))
            self.schedule.add(r)

        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [70]:
M = 10
N = 10
numAgentes = 5
porcCeldasSucias = 0.6
tiempoMaximo = 1

model = Habitacion(M,N,numAgentes,porcCeldasSucias,tiempoMaximo)
startTime = time.time()
while((time.time() - startTime) < tiempoMaximo):
    model.step()

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - startTime))))

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(all_grid.to_string())